# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from math import log

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = pd.read_csv('kc_house_data.csv')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. Make sure to add "normalize=True" when creating the Lasso object. Refer to the following code snippet for the list of features.

In [5]:
#model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
#                                                validation_set=None, 
#                                                l2_penalty=0., l1_penalty=1e10)

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, normalize=True)

Find what features had non-zero weight.

In [6]:
model_all.coef_

array([    0.        ,     0.        ,     0.        ,   134.43931396,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        , 24750.00458561,     0.        ,
       61749.10309071,     0.        ,     0.        ,    -0.        ,
           0.        ])

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

[ sqft_living , view , grade]

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
testing = pd.read_csv('wk3_kc_house_test_data.csv')
training = pd.read_csv('wk3_kc_house_train_data.csv')
validation = pd.read_csv('wk3_kc_house_valid_data.csv')

In [8]:
training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

In [9]:
validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

In [10]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [11]:
rss_on_validation = []
for l1_penalty in np.logspace(1, 7, num=13):
    
    lm = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model = lm.fit(training[all_features], training['price'])
    prediction = model.predict(validation[all_features])
   
    errors = prediction - validation['price']
    rss_on_validation.append((errors * errors).sum())
    print("--------------------------------")
    print((errors * errors).sum())
    print(l1_penalty)
min(rss_on_validation)

--------------------------------
398213327300134.9
10.0
--------------------------------
399041900253346.94
31.622776601683793
--------------------------------
429791604072559.6
100.0
--------------------------------
463739831045121.1
316.22776601683796
--------------------------------
645898733633800.8
1000.0
--------------------------------
1222506859427163.0
3162.2776601683795
--------------------------------
1222506859427163.0
10000.0
--------------------------------
1222506859427163.0
31622.776601683792
--------------------------------
1222506859427163.0
100000.0
--------------------------------
1222506859427163.0
316227.7660168379
--------------------------------
1222506859427163.0
1000000.0
--------------------------------
1222506859427163.0
3162277.6601683795
--------------------------------
1222506859427163.0
10000000.0


398213327300134.9

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`? 10

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have? 15

In [12]:
model_all = linear_model.Lasso(alpha=10, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights
model_all.coef_

array([-1.91323282e+04,  1.50903360e+02,  4.49977814e+04,  4.77489901e+02,
       -3.13239406e+04,  3.35704021e-01, -4.40352566e+02, -0.00000000e+00,
        5.99894130e+03,  5.55888903e+05,  4.25214762e+04,  2.34009683e+04,
        1.28004811e+05,  0.00000000e+00,  3.08686401e+00, -3.19759645e+03,
        1.34355567e+01])

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [15]:
nz_list = []
for l1_penalty in  np.logspace(1, 4, num=20):
   
    lm = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model = lm.fit(training[all_features], training['price'])
    nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    nz_list.append(nonzeros)
    print(l1_penalty)
nz_list

10.0
14.38449888287663
20.6913808111479
29.76351441631318
42.81332398719393
61.58482110660264
88.58667904100822
127.42749857031335
183.29807108324357
263.6650898730358
379.26901907322497
545.5594781168514
784.7599703514607
1128.8378916846884
1623.776739188721
2335.7214690901214
3359.818286283781
4832.930238571752
6951.927961775606
10000.0


[15, 15, 15, 15, 13, 12, 11, 10, 7, 6, 6, 6, 5, 3, 3, 2, 1, 1, 1, 1]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [16]:
l1_penalty_min = 127.42749857
l1_penalty_max = 263.66508987 

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [17]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [18]:
validation_rss = []
sparsity = []
l1_penalty_list = []

for l1_penalty in l1_penalty_values:

    l1_penalty_list.append(l1_penalty)

    lm = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model = lm.fit(training[all_features], training['price'])
    
    nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    sparsity.append(nonzeros)
    
    prediction = model.predict(validation[all_features])
    errors = prediction - validation['price']
    
    rss = (errors * errors).sum()
    validation_rss.append(rss)

In [19]:
df = pd.DataFrame({'l1_penalty':l1_penalty_list,'sparsity':sparsity,'validation_rss':validation_rss})
df.sort_values(by=['validation_rss'])

,l1_penalty,sparsity,validation_rss
0,127.427499,10,4.353747e+14
1,134.597898,10,4.370092e+14
2,141.768298,8,4.382361e+14
3,148.938697,8,4.391589e+14
4,156.109097,7,4.400374e+14
5,163.279496,7,4.407775e+14
6,170.449896,7,4.415667e+14
7,177.620295,7,4.424064e+14
8,184.790695,7,4.432967e+14
9,191.961094,7,4.442398e+14


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [20]:
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
# Learn a model on TRAINING data using the specified l1_penalty.
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_5 = model.fit(training[all_features], training['price'])
    nonzeros = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
    print(l1_penalty)
    print(nonzeros)

127.42749857
10
134.59789811210527
10
141.76829765421053
8
148.9386971963158
8
156.10909673842104
7
163.2794962805263
7
170.44989582263156
7
177.62029536473682
7
184.79069490684208
7
191.96109444894734
7
199.1314939910526
7
206.30189353315785
6
213.4722930752631
6
220.64269261736837
6
227.81309215947365
6
234.9834917015789
6
242.15389124368417
6
249.32429078578943
6
256.4946903278947
6
263.66508987
6


In [21]:
l1_penalty = 156.10909
lm = linear_model.Lasso(alpha=l1_penalty, normalize=True)
model = lm.fit(training[all_features], training['price'])
model.coef_

array([-0.00000000e+00, -0.00000000e+00,  1.06108917e+04,  1.63380251e+02,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  5.06451692e+05,  4.19600437e+04,  0.00000000e+00,
        1.16253554e+05,  0.00000000e+00,  0.00000000e+00, -2.61223493e+03,
        0.00000000e+00])

In [23]:
all_features

['bedrooms',
 'bedrooms_square',
 'bathrooms',
 'sqft_living',
 'sqft_living_sqrt',
 'sqft_lot',
 'sqft_lot_sqrt',
 'floors',
 'floors_square',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?  156.109097
2. What features in this model have non-zero coefficients? [bathrooms, sqft_living, waterfront, view, grade, yr_built] 